Pranav Gupta, prannerta100@gmail.com


General: I used tkinter, Python3 to write my code. Running the required problem snippet results in a Tk window.

Problem 1: highlight points

Problem 2: select a bunch of points, create a best fit circle

Problem 3: select a bunch of points, create a best fit ellipse


Problem 1:

Here when you run the snippet, you see a 20x20 grid of grey dots. Click and drag to create a circle. After you let go of the mouse left button, points along the boundary will get automatically highlighted. 3 circles are drawn, the best fit circle being boldface. The max and min radius circles are the other 2 circles.

Double click clears things out, be careful!

In [0]:
from tkinter import *
import math
trace = 0 
#basic structure based on http://www.java2s.com/Code/Python/GUI-Tk/Usemousetodrawashapeoncanvas.htm 

class CanvasEventsDemo: 
    def __init__(self, parent=None):
        #grid spacing hard coded to 20 pixels
        self.gridsp = 20
        #will make a 20x20 grid
        self.gridsize = 20
        #width and height of the canvas
        self.size = (self.gridsize - 1) * self.gridsp
        #create canvas
        canvas = Canvas(width=self.size, height= self.size, bg='beige') 
        rnge = range(1,self.size + 2, self.gridsp) #pixel positions on the screen, for x and y coords of the dots
        for i in rnge:
            for j in rnge:
                canvas.create_oval(i-2, j+2, i+2, j-2, fill = 'grey')
                #create oval with a 4 pixel diameter
        canvas.pack()
        #create bindings with the appropriate mouse buttons
        canvas.bind('<ButtonPress-1>', self.onStart) 
        canvas.bind('<ButtonRelease-1>', self.onRelease) 
        canvas.bind('<B1-Motion>', self.onGrow)  
        canvas.bind('<Double-Button-1>', self.onClear)   
        self.canvas = canvas
        self.drawn  = None
        
    def onStart(self, event):
        self.start = event
        self.drawn = None
        
    def onGrow(self, event):
        #redefine canvas
        canvas = event.widget
        #delete the previous oval, draw the new oval as per the new mouse position 
        if self.drawn: 
            canvas.delete(self.drawn)
        #calculate the radius of the circle, based on initial and final positions of the mouse (onStart --> onGrow)
        self.rad = (abs(self.start.x-event.x) + abs(self.start.y-event.y))/2 #force to draw a circle only, but could draw ellipses in general
        #create the circle, create_oval needa coordinates of the top left and the bottom right corners
        objectId = canvas.create_oval(self.start.x-self.rad,self.start.y+self.rad,self.start.x+self.rad,self.start.y-self.rad, width = 3)
        #print objID if trace is known
        if trace: 
            print(objectId)
        #tell everyone that an oval has been drawn, when the oval grows, delete the previous oval
        self.drawn = objectId
    
    def onRelease(self, event):
        #when you release the mouse left button, you need to finalise the oval, highlight points close to the oval
        canvas = event.widget #can now write canvas...., instead of event.widget..... Short and easy-to-read alias for event.widget
        
        if self.drawn != None:
            #time to color the close-by guys on the 20x20 grid
            #print(self.rad)
            normalized_radius = self.rad/self.gridsp #this is not an integer, just the radius in units of gridspacing
            #need to calculate radii for max and min circles too, so as points are added , I will compare to the current max and min rad values, update as needed
            maxrad = self.rad
            minrad = self.rad
            #the initial coordinates, that is, the first point on the grid being made blue
            nx0 = nearest_integer((self.start.x+self.rad-1)/self.gridsp)
            ny0 = nearest_integer((self.start.y-1)/self.gridsp)
            #now start coloring
            nx = nx0
            ny = ny0
            xcent = 1 + nx * self.gridsp
            ycent = 1 + ny * self.gridsp
    
            cnt = 0 #counts how many points colored, update for each loop iteration by +1
            quad = 11 #flag, first digit tells if nx = nx0, second digit tells if ny = ny0. This helps us end the circle when you come back to the starting point.
            
            #algorithm: depending on the quadrant, move 1 step right/left, 1 step down/up or 1 step along 45, 135, 270, 335 degs
            #So 8 possible moves 
            # + + +
            # + 0 +
            # + + +
            #0 curr position, +: one of the 8 prospective next positions
            #keep looping till you reach back at (nx0,ny0) (or quad becomes equal to 11)
            while cnt == 0 or quad != 11:
                #know your quadrant, del_x (y) : change in x (y) coordinate w.r.t. center. Sign of del_x(y) tells the available ways to move
                #CAUTION: vertically down is + Y axis for the canvas, right is +X, left is -X, up is -Y
                del_x = (1 + nx * self.gridsp - self.start.x)/self.gridsp
                del_y = (1 + ny * self.gridsp - self.start.y)/self.gridsp
                
                #depending on the quadrant, you have 3 options (you don't want to step away from the circle, hence you can always eliminate 5 out of the 8 possible moves)
                if del_y >= 0:
                    if del_x >= 0: #for example, left, down, left-down are the 3 meanningful options
                        #choose the one that takes you closer to the locus of the circle
                        #typically only one of the 3 options will be the best, hence didn't bother taking a minimum of the 3 options and choosing it as the next step
                        if abs(math.sqrt(del_x ** 2 + (del_y + 1) ** 2) - normalized_radius) < 0.5:
                            ny = ny + 1
                        elif abs(math.sqrt((del_x - 1) ** 2 + (del_y + 1) ** 2) - normalized_radius) < 0.5:
                            nx = nx - 1
                            ny = ny + 1
                        else:#abs(math.sqrt((del_x - 1) ** 2 + del_y ** 2) - normalized_radius) < 0.5: 
                            nx = nx - 1
                    if del_x < 0: #
                        if abs(math.sqrt((del_x - 1) ** 2 + del_y ** 2) - normalized_radius) < 0.5: 
                            nx = nx - 1
                        elif abs(math.sqrt((del_x - 1) ** 2 + (del_y - 1) ** 2) - normalized_radius) < 0.5:
                            nx = nx - 1
                            ny = ny - 1
                        else: #abs(math.sqrt(del_x ** 2 + (del_y - 1) ** 2) - normalized_radius) < 0.5:
                            ny = ny - 1
                else:
                    if del_x < 0:
                        if abs(math.sqrt(del_x ** 2 + (del_y - 1) ** 2) - normalized_radius) < 0.5:
                            ny = ny - 1
                        elif abs(math.sqrt((del_x + 1) ** 2 + (del_y - 1) ** 2) - normalized_radius) < 0.5:
                            nx = nx + 1
                            ny = ny - 1
                        else:#abs(math.sqrt((del_x + 1) ** 2 + del_y ** 2) - normalized_radius) < 0.5:
                            nx = nx + 1
                    if del_x >= 0:
                        if abs(math.sqrt((del_x + 1) ** 2 + del_y ** 2) - normalized_radius) < 0.5:
                            nx = nx + 1
                        elif abs(math.sqrt((del_x + 1) ** 2 + (del_y + 1) ** 2) - normalized_radius) < 0.5:
                            nx = nx + 1
                            ny = ny + 1
                        else: #abs(math.sqrt(del_x ** 2 + (del_y + 1) ** 2) - normalized_radius) < 0.5:
                            ny = ny + 1
                #we calculated the new position, now make the dot blue!            
                xcent = 1 + nx * self.gridsp
                ycent = 1 + ny * self.gridsp
                #update minrad and maxrad
                dist = math.sqrt((xcent - self.start.x) ** 2 + (ycent - self.start.y) ** 2)
                if(minrad > dist):
                    minrad = dist
                if(maxrad < dist):
                    maxrad = dist
                #highlight the new position blue
                canvas.create_oval(xcent - 2, ycent + 2, xcent + 2, ycent - 2, fill = 'blue')
                #update count
                cnt += 1
                #check flag, if quad = 11, then 1 rotation over, no point looping further, quit the while loop
                quad = 10 * (nx == nx0) + (ny == ny0) #quad = 11 means that (nx = nx0) and (ny = ny0), so back to origin
            
            #print(quad) #: diagnostic
            
            #create the min rad and max rad circles
            canvas.create_oval(self.start.x - minrad, self.start.y + minrad, self.start.x + minrad, self.start.y - minrad)
            canvas.create_oval(self.start.x - maxrad, self.start.y + maxrad, self.start.x + maxrad, self.start.y - maxrad)
            #print(cnt) : diagnostic, to know how many points highlighted
    def onClear(self, event):
        #clears the screen
        event.widget.delete('all')                   

            
            
def nearest_integer(x):
    #this function helps you choose the nearest grid point
    #math.floor is not enough; math.floor(5.8) = 5, but I want 6. So I defined my own function
    y = math.floor(x)
    if x - y > 0.5:
        y = y + 1
    return y

#main loop, creates the Tk window
if __name__ == '__main__':
    CanvasEventsDemo()
    mainloop()

Problem 2: Best fit circle

A proper fit to the highlighted points challenging, as you need to solve a non-linear least squares fit problem. Faster to use the following algorithm (didn't copy it from anywhere, my own):

Double click clears things out, be careful!

Take all the x coordinates, all the y coordinates. Then vector average over all of them. What you get is an estimate for the coordinates of the center of the circle. 

Getting the radius: now that you know the center, average over all the Euclidean distances to all the points, that's the estimate for the radius.

Works like the previous widget, single click on point to include it, click again to remove it; when selected, the point appears blue



In [0]:
from tkinter import *
import math
trace = 0 

class CanvasEventsDemo: 
    def __init__(self, parent=None):
        self.gridsp = 20
        self.gridsize = 20
        self.size = (self.gridsize - 1) * self.gridsp
        #just like Problem 1, create some extra height at the bottom, to fit the generate button
        canvas = Canvas(width=self.size, height= self.size + 30, bg='beige') 
        #bindings
        canvas.bind('<ButtonPress-1>', self.onRelease) 
        canvas.bind('<Double-Button-1>', self.onClear)   #Double-Button-1 not ideal
        #plot the grid on the screen
        rnge = range(1,self.size + 2, self.gridsp)
        for i in rnge:
            for j in rnge:
                canvas.create_oval(i-2, j+2, i+2, j-2, fill = 'grey')
        #store canvas in the self.canvas variable
        self.canvas = canvas
        #this list stores all the tuples of x,y coordinates of teh highlighted points
        self.coord_list = []
        #create the generate button, don't know why the name "generate" doesn't show up
        GenButton = Button(canvas, width=10, height=15, borderwidth=5, text = "generate", command = lambda: self.onGenerate(), bg="red")
        #place the generate button at the bottom of the screen
        GenButton.place(x=self.size/2 - 5, y=self.size + 10)
        canvas.pack()
        
    def onRelease(self, event):
        #when button is released, add point to the list, highlight it
        canvas = event.widget
        #calculate the nearest point on the grid
        nx = nearest_integer((event.x-1)/self.gridsp)
        ny = nearest_integer((event.y-1)/self.gridsp)
        #calculate the coordinate of the closest grid point
        xcent = 1 + nx * self.gridsp
        ycent = 1 + ny * self.gridsp
        #toggle off if point was already present in the list, color will return to grey after removing the point from the list
        if (xcent,ycent) in self.coord_list:
            #toggle off
            #discard from the list
            self.coord_list.remove((xcent, ycent))
            canvas.create_oval(xcent-2, ycent+2, xcent+2, ycent-2, fill = 'grey')
        else:
            #toggle on
            #new entry into self.coords_list, welcome it in by appending, make it blue
            self.coord_list.append((xcent, ycent))
            canvas.create_oval(xcent-2, ycent+2, xcent+2, ycent-2, fill = 'blue')
            
        
    def onClear(self, event):
        #clear all the highlighted points, leaves only the best fit circle
        #I like it that way because we have traces of the history of the widget, only the list of points is emptied
        #print(self.coord_list): #diagnostic
        #make each point return to grey
        for c in self.coord_list:
            xcent = c[0]
            ycent = c[1]
            self.canvas.create_oval(xcent-2, ycent+2, xcent+2, ycent-2, fill = 'grey')
        #now empty the list, screen restored, we are back to the initial state of the widget
        self.coord_list = []
            
    def onGenerate(self):
        #refer to the lambda function, this is the command executed when the generate button at the bottom is pressed
        #print("Hello") #diagnostic
        #print(self.coord_list) #diagnostic
        
        #return nothing if no list, otherwise division by 0 will happen
        if len(self.coord_list) == 0:
            return None
        #calculate the mean of all coordinates, to know the center of the circle
        xsum = 0
        ysum = 0
        #didn't use numpy
        for coord in self.coord_list:
            xsum += coord[0]
            ysum += coord[1]
        #(xcent,ycent) is the center of the best fit circle
        xcent = xsum / len(self.coord_list)
        ycent = ysum / len(self.coord_list)
        #now average over all distances from (xcent,ycent)
        tmp = 0
        for coord in self.coord_list:
            tmp += math.sqrt((coord[0] - xcent) ** 2 + (coord[1] - ycent) ** 2)
        rad = tmp / len(self.coord_list)
        #now draw the best fit circle
        self.canvas.create_oval(xcent-rad, ycent+rad, xcent+rad, ycent-rad)
        #done
        self.rad = rad #just store rad into self.rad, just in case
        #print(self.rad)         #diagnostic
    
#nearest integer function, as in Problem 1
def nearest_integer(x):
    y = math.floor(x)
    if x - y > 0.5:
        y = y + 1
    return y

#run the widget
if __name__ == '__main__':
    CanvasEventsDemo()
    mainloop()

Problem 3: Ellipse

Here the problem is again a non-linear least squares fit problem, and we are optimizing for not just 1 radius, but 2 radii (major/minor axis).

Double click clears things out, be careful!

I am focusing on ellipses of the form $$\frac{(x-x_0)^2}{a^2} + \frac{(y-y_0)^2}{b^2} = 1$$.

The first step of creating a best fit ellipse is similar to my algorithm in Problem 2. Average over x and y coordinates separately, this gives you an estimate for the center of the ellipse, just like the center of the circle. 

**Calculating the major and minor axes:**

I was inspired from C vanLoan's ppt (https://www.cs.cornell.edu/cv/OtherPdf/Ellipse.pdf).

Rewrite $$\frac{(x-x_0)^2}{a^2} + \frac{(y-y_0)^2}{b^2} = 1$$ as $$(x-x_0)^2 + D (y-y_0)^2 = E$$.
Do a least squares fit minimization of $(x-x_0)^2 + D (y-y_0)^2 = E$, by defining your residual as $((x-x_0)^2 + D (y-y_0)^2 - E)^2$, summed over all points $(x,y)$. This results in minimizing $||Aw-b||^2$, where $A = [[y_1^2, -1],[y_2^2, -1], ...]^T$, $b = [-x_1^2, -x_2^2, ...]^T$, and $w = [D,E]^T$.

The answer is $w = [D, E]^T = (A^T A)^{-1} A^T b$. Once you get D and E, $a = \sqrt{E}$, $b = \sqrt{E/D}$.

Ellipse tilt is not taken into account here. More work, but possible. You could fit a more general ellipse equation $x^2 + B y^2 + C x y + D x + E y + F = 0$ and then plot it to get any general ellipse ($B^2-4AC <0$ is a constraint, however). The A and b matrices will look different, but the same method will work. 

Everything in the following widget is like Problem 2, only that we generate ellipses instead of circles.



In [0]:
from tkinter import *
import math
import numpy as np
trace = 0 

class CanvasEventsDemo: 
    def __init__(self, parent=None):
        self.gridsp = 20
        self.gridsize = 20
        self.size = (self.gridsize - 1) * self.gridsp
        #just like Problem 1, create some extra height at the bottom, to fit the generate button
        canvas = Canvas(width=self.size, height= self.size + 30, bg='beige') 
        #bindings
        canvas.bind('<ButtonPress-1>', self.onRelease) 
        canvas.bind('<Double-Button-1>', self.onClear)   #Double-Button-1 not ideal
        #plot the grid on the screen
        rnge = range(1,self.size + 2, self.gridsp)
        for i in rnge:
            for j in rnge:
                canvas.create_oval(i-2, j+2, i+2, j-2, fill = 'grey')
        #store canvas in the self.canvas variable
        self.canvas = canvas
        #this list stores all the tuples of x,y coordinates of teh highlighted points
        self.coord_list = []
        #create the generate button, don't know why the name "generate" doesn't show up
        GenButton = Button(canvas, width=10, height=15, borderwidth=5, text = "generate", command = lambda: self.onGenerate(), bg="red")
        #place the generate button at the bottom of the screen
        GenButton.place(x=self.size/2 - 5, y=self.size + 10)
        canvas.pack()
        
    def onRelease(self, event):
        #when button is released, add point to the list, highlight it
        canvas = event.widget
        #calculate the nearest point on the grid
        nx = nearest_integer((event.x-1)/self.gridsp)
        ny = nearest_integer((event.y-1)/self.gridsp)
        #calculate the coordinate of the closest grid point
        xcent = 1 + nx * self.gridsp
        ycent = 1 + ny * self.gridsp
        #toggle off if point was already present in the list, color will return to grey after removing the point from the list
        if (xcent,ycent) in self.coord_list:
            #toggle off
            #discard from the list
            self.coord_list.remove((xcent, ycent))
            canvas.create_oval(xcent-2, ycent+2, xcent+2, ycent-2, fill = 'grey')
        else:
            #toggle on
            #new entry into self.coords_list, welcome it in by appending, make it blue
            self.coord_list.append((xcent, ycent))
            canvas.create_oval(xcent-2, ycent+2, xcent+2, ycent-2, fill = 'blue')
            
        
    def onClear(self, event):
        #clear all the highlighted points, leaves only the best fit circle
        #I like it that way because we have traces of the history of the widget, only the list of points is emptied
        #print(self.coord_list): #diagnostic
        #make each point return to grey
        for c in self.coord_list:
            xcent = c[0]
            ycent = c[1]
            self.canvas.create_oval(xcent-2, ycent+2, xcent+2, ycent-2, fill = 'grey')
        #now empty the list, screen restored, we are back to the initial state of the widget
        self.coord_list = []
            
    def onGenerate(self):
        #refer to the lambda function, this is the command executed when the generate button at the bottom is pressed
        #print("Hello") #diagnostic
        #print(self.coord_list) #diagnostic
        
        #return nothing if no list, otherwise division by 0 will happen
        if len(self.coord_list) == 0:
            return None
        #calculate the mean of all coordinates, to know the center of the circle
        xsum = 0
        ysum = 0
        #didn't use numpy
        for coord in self.coord_list:
            xsum += coord[0]
            ysum += coord[1]
        #(xcent,ycent) is the center of the best fit circle
        xcent = xsum / len(self.coord_list)
        ycent = ysum / len(self.coord_list)
        #now average over all distances from (xcent,ycent)
       
        tmp = 0
        #define A, an nx2 matrix; using numpy for matrix stuff
        A = np.zeros((len(self.coord_list),2))
        #define B, an nx1 matrix
        b = np.zeros((len(self.coord_list),1))
        #minimize (Ax-b)^2, as described earlier
        
        #put in values for A, b - explained in the description
        for i in range(len(self.coord_list)):
            A[i][0] = (self.coord_list[i][1]-ycent)**2
            A[i][1] = -1
            b[i][0] = - (self.coord_list[i][0]-xcent)**2
        #solves the ||Ax-b|| minimization problem, similar to c vanLoan's pdf
        sol = np.linalg.solve(np.matmul(A.transpose(),A),np.matmul(A.transpose(),b))
        #print(sol) #diagnostic
        E = sol[1][0]
        D=sol[0][0]
        #we have D, E, now calculate a and b in the ellipse equation
        #this is an (x-x0)^2/a^2+(y-y0)^2/b^2 = 1 style ellipse
        self.axX = math.sqrt(E) #a
        self.axY = math.sqrt(E/D) #b
        #plot the ellipse
        self.canvas.create_oval(xcent - self.axX, ycent + self.axY, xcent + self.axX , ycent - self.axY)
        #done
        #print(self.axX, self.axY)         #diagnostic
    
#nearest integer function, as in Problem 1
def nearest_integer(x):
    y = math.floor(x)
    if x - y > 0.5:
        y = y + 1
    return y

#run the widget
if __name__ == '__main__':
    CanvasEventsDemo()
    mainloop()